In [1]:
# Guessing this is where we will be saving the extrcated features, but not very sure yet. Let's follow along the notebook, and see where it leads us to.

In [22]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from data_generator import DataGenerator, batch_predict

In [3]:
# load the model?

In [4]:
overlapping_model = tf.keras.models.load_model("models/CNN_seq2seq_overlapping.keras") 

2025-08-28 13:19:48.639901: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [5]:
overlapping_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 13, None, None, 3)   0         []                            
                             ]                                                                    
                                                                                                  
 time_distributed (TimeDist  (None, 13, 32)               71808     ['input_1[0][0]']             
 ributed)                                                                                         
                                                                                                  
 lstm (LSTM)                 [(None, 64),                 24832     ['time_distributed[0][0]']    
                              (None, 64),                                                   

In [6]:
# locate the test data?

In [7]:
# input features
input_features_loc = 'data/test_input_sub_images/'
input_contents = os.listdir(input_features_loc)
input_contents.sort()

In [8]:
input_contents

['test_data_blk_0103.npy',
 'test_data_blk_0104.npy',
 'test_data_blk_0105.npy',
 'test_data_blk_0106.npy',
 'test_data_blk_0201.npy',
 'test_data_blk_0202.npy',
 'test_data_blk_0205.npy',
 'test_data_blk_0206.npy',
 'test_data_blk_0302.npy',
 'test_data_blk_0303.npy',
 'test_data_blk_0304.npy',
 'test_data_blk_0305.npy',
 'test_data_blk_0306.npy']

In [9]:
# Running inot a lot of memory issues, therefore the loop is cut into chunks to get and save the required outputs.

In [10]:
%%time
for_sanity_check = []
for i in range(6, len(input_contents)):
    # load the np file
    load_np_file = np.load(os.path.join(input_features_loc, input_contents[i]))
    # print shape of the loaded file
    print(load_np_file.shape)
    # predicted_values
    # define the test data generator here
    test_gen = DataGenerator(feature1, target, batch_size=32, shuffle=False)
    predicted_values = overlapping_model.predict(load_np_file)
    print(predicted_values.shape)
    for_sanity_check.append(predicted_values)
    # save these values?
    # name
    loc_name = 'data/predicted_sequences_from_stage_1/' + 'pred_values_blk_' + input_contents[i].split('.')[0][-4:] + '.npy'
    np.save(loc_name, predicted_values)

(3072, 13, 100, 100, 3)


2025-08-28 13:20:11.860649: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


96/96 [==============================] - 6s 27ms/step
(3072, 7, 32)
(3072, 13, 100, 100, 3)
96/96 [==============================] - 3s 27ms/step
(3072, 7, 32)
(3072, 13, 100, 100, 3)
96/96 [==============================] - 3s 28ms/step
(3072, 7, 32)
(3072, 13, 100, 100, 3)
96/96 [==============================] - 3s 27ms/step
(3072, 7, 32)
(3072, 13, 100, 100, 3)
96/96 [==============================] - 3s 28ms/step
(3072, 7, 32)
(3072, 13, 100, 100, 3)
96/96 [==============================] - 3s 28ms/step
(3072, 7, 32)
(3072, 13, 100, 100, 3)
96/96 [==============================] - 3s 28ms/step
(3072, 7, 32)
CPU times: user 1min 7s, sys: 56.4 s, total: 2min 3s
Wall time: 2min 50s


In [11]:
# might need to do a batch predict due to the dataset size, let's figure this out - okay, why don't we actually do this and try to write a cleaner code? That might be better for future refernces.

In [12]:
# Nah let it be for now, it would take some time - we could get all the extracted features saved in two runs, I think it's okay for now

In [13]:
# Okay, we have done another sanity check here, which i believe is not necessary, but let's try it anyway until we get a memory error

In [14]:
# perform a sanity check
loc_path = 'data/predicted_sequences_from_stage_1/'

In [16]:
loc_contents = os.listdir(loc_path)
loc_contents.sort()

In [17]:
loc_contents

['pred_values_blk_0103.npy',
 'pred_values_blk_0104.npy',
 'pred_values_blk_0105.npy',
 'pred_values_blk_0106.npy',
 'pred_values_blk_0201.npy',
 'pred_values_blk_0202.npy',
 'pred_values_blk_0205.npy',
 'pred_values_blk_0206.npy',
 'pred_values_blk_0302.npy',
 'pred_values_blk_0303.npy',
 'pred_values_blk_0304.npy',
 'pred_values_blk_0305.npy',
 'pred_values_blk_0306.npy']

In [23]:
# Okay, for this we do need the entire loop to run to get the predicted values - so let's just do the sanity check starting from the 6th file here

In [29]:
%%time
for i in range(6, len(loc_contents)):
    load_stored_preds = np.load(os.path.join(loc_path, loc_contents[i]))
    print(np.mean(load_stored_preds == for_sanity_check[i-6]))

1.0
1.0
1.0
1.0
1.0
1.0
1.0
CPU times: user 12.2 ms, sys: 11.3 ms, total: 23.5 ms
Wall time: 21.7 ms


In [20]:
pred_values = np.load(os.path.join(loc_path, loc_contents[0]))

In [21]:
pred_values.shape

(3072, 7, 32)

In [30]:
# I think then we have examined the extracted features against the true values - so let's continue this work after food?